In [1]:
!pip install gradio
!pip install transformers
!pip install torch
!pip install google-cloud-storage

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 837.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 6.9 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing 

In [2]:
import gradio as gr
from transformers import pipeline
import numpy as np
import spacy
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img

In [3]:
transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-base.en")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.94k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/290M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.83k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

In [4]:
# Load the spaCy model (needed to extract name from a sentence)
nlp = spacy.load('en_core_web_sm')

In [5]:
def extract_name(text):
    """
    Function to extract name from a sentence

    Args:
      text: Spoken Text string
    Returns:
      Name
    """
    doc = nlp(text)
    name = [ent.text for ent in doc.ents if ent.label_ == 'PERSON']
    if name:
      return name[0]
    else:
      "No name found"

In [6]:
# # Function to Translate Speech to text & return return patient's name
# def transcribe(audio):
#     sr, y = audio
#     y = y.astype(np.float32)
#     y /= np.max(np.abs(y))

#     text = transcriber({"sampling_rate": sr, "raw": y})["text"]
#     return text


In [7]:
from PIL import Image
import requests
from io import BytesIO

# Get Image using patient's name from GCS
def image_from_gcs(name):
  try:
    # URL of the image
    gcs_file_url = 'https://storage.googleapis.com/gh_patient_images/'+ name +'.jpg'

    # Fetch the image from GCS
    response = requests.get(gcs_file_url)
    image = Image.open(BytesIO(response.content))
    return image
  except Exception as e:
      return ("Error reading image file:", str(e))

In [8]:
def prepare_image(image):
    """
    Prepare image for model
    Args:
      image: Get patient's image
    Steps:
    - Convert Image to numpy file
    - COnvert to floating point and normalize the file

    Returns: Normalized image file

    """
    # Resize image to 300x300
    image = tf.image.resize(image, (300, 300))

    # Normalize image
    image_normalized = np.array(image).astype(np.float32) / 255

    # Add batch dimension
    image_normalized = np.expand_dims(image_normalized, axis=0)  # Shape becomes (1, 300, 300, 3)

    return image_normalized

In [9]:
# Get model's prediction
def prediction(name, image_normalized):
    # Load the model
    try:
        # Path to model in GCS
        gcs_model_path = 'gs://gh_gcs_bucket/melanoma_cnn_model.keras'
        # Load the model directly from GCS
        model = tf.keras.models.load_model(gcs_model_path)
    except Exception as e:
        return ("Error reading model:", str(e))

    # Model Prediction
    try:
      prediction_probs = model.predict(image_normalized)
    except Exception as e:
      return f"Error during prediction: {e}"

    prediction = np.argmax(prediction_probs, axis=1)
    confidence = np.max(prediction_probs)

    # Interpret the prediction
    if prediction == 1:
        return f"Our model's prediction for {name} is malignant with {confidence*100:.2f}% confidence."
    else:
        return f"Our model's prediction for {name} is benign with {confidence*100:.2f}% confidence."

In [10]:
# Use audio to get model's prediciton
def process_speech(audio):
  """
  Input = Audio
  Output = Prediction text string
  """
  # 1. Speech to text conversion and get patient name
  # name = transcribe(audio)
  sr, y = audio
  y = y.astype(np.float32)
  y /= np.max(np.abs(y))

  text = transcriber({"sampling_rate": sr, "raw": y})["text"]

  #Get name from text
  name = extract_name(text)
  if name == "No name found":
    return "No name found"

  # 2. Get Image from GCS file using Patient's name
  try:
    image = image_from_gcs(name)
  except Exception:
    return "No image content found."

  # 3. Prepare image for model prediction
  image_normalized = prepare_image(image)

  # 4. Get CNN Model's prediction
  results = prediction(name, image_normalized)
  return results, image

In [11]:
# Gradio app
onco_assistant_app = gr.Interface(
    fn=process_speech,                # Main function to process speech
    inputs = gr.Audio(sources="microphone"),  # Audio input
    outputs = [gr.Textbox(label="Prediction Results", lines=2),   # Text output
              gr.Image(type="pil", label="Analyzed Image", visible=True)],  # Image output

    # inputs=gr.Audio(sources=["microphone"]),
    # outputs=["text"],
    title="Physician's Assistant for Melanoma Detection",
    description="Please ask for results of one the these patient's sample images - Marie, Benjamin, Blake, Brooke, Maya & Mario"

)

In [12]:
# Launch App
onco_assistant_app.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://a8777179ea129e3418.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
